In [1]:
import pandas as pd # loadingData
from nltk.stem import PorterStemmer # func1,cleanData
from nltk.corpus import stopwords # func1
import regex as re # func1
import nlpaug.augmenter.word as naw #func5 CreateOversamplingWithDataAugmentation
import gc # func5
# from keras.preprocessing.text import Tokenizer #func2 SplitTrainTestDataForMultiLabelWithDataAugmentation && func4 Tokenize
# from tensorflow.keras.preprocessing.sequence import pad_sequences #func4 Tokenize
import numpy as np # (*func7*) one_hot_encode

# main -> 1, 2, 3
    # (*func1*) Cleandata
    # (*func2*) SplitTrainTestDataForMultiLabelWithDataAugmentation -> 5, 6, 8
        # (*func5*) CreateOversamplingWithDataAugmentation
        # (*func6*) create_tag_mapping -> 7
            # (*func7*) one_hot_encode
        # (*func8*) RemoveTestRecordIfNotExistInTrainData
    # (*func3*) SplitTrainTestData
    # (*func4*) Tokenize


In [10]:
# Setup Project Parameters
DataAugmentation, DataAugThreshold = True, 30000
DataFilePath, DataFileName, FileType = "/Users/lycoris/Desktop/innovation/multitriage/Data/", "Issueazure-powershellWebScrap", ".csv"
MAX_SEQUENCE_LENGTH = 300
EMBEDDING_DIM = 100
LoadDataAugFromFile = False
LearningRate = 0.001

In [11]:
import pandas as pd
# Loading Data
# 读取Datafile, 返回删除格式错误的行，不用index列(不知为啥)，dtype=unicode，编码方式latin-1，low-memory避免数据类型不同的问题
# sample(frac = 1)打乱取全部(不懂，后面都按CreateDate排序了)
totalData = pd.read_csv(DataFilePath + DataFileName + FileType, error_bad_lines=False, index_col=False, dtype='unicode',
                        encoding='latin-1', low_memory=False).sample(frac=1)
# 替换RepoID列，不在源文件上改动(IssueaspnetcoreWebScrap的列RepoID本来就是RepoID了)
totalData = totalData.rename(columns={'ï»¿RepoID': 'RepoID'}, inplace=False)
# 04/09/2017  -> 2014-01-07  
totalData['CreatedDate'] = pd.to_datetime(totalData['CreatedDate'])
totalData = totalData.sort_values(by=['CreatedDate'])
totalData

,RepoID,IssueID,Title_Description,AST,FixedByID,Name,CreatedDate
1094,azure-powershellWebScrap,558,new-azurereservedip no longer working with pow...,NaN,unknown,network| service attention,2015-01-07
2153,azure-powershellWebScrap,832,support medialink in set-azureplatformextensio...,NaN,singhkays,compute| service attention,2015-01-09
1271,azure-powershellWebScrap,1386,get-azurermnetworksecuritygroup erroraction is...,NaN,unknown,network| service attention,2015-01-12
1268,azure-powershellWebScrap,1380,setting reversefqdn in new-azurermpublicipaddr...,NaN,unknown,network| service attention,2015-01-12
1270,azure-powershellWebScrap,1385,get-azurevm fails with unable to find type [mi...,NaN,unknown,unknown,2015-01-12
...,...,...,...,...,...,...,...
1570,azure-powershellWebScrap,11311,az.recoveryservices specifically asr cmdlets s...,NaN,sharmistha-rai,recovery services| recovery services site-reco...,2020-12-03
2229,azure-powershellWebScrap,11882,update-azwebappaccessrestrictionconfig -scmsit...,NaN,unknown,app services| service attention| question,2020-12-05
2230,azure-powershellWebScrap,11885,get-azaks is missing (many) details provided b...,NaN,erich-wang,aks| azure ps team| feature-request,2020-12-05
13,azure-powershellWebScrap,12135,set-aznetworkwatcherconfigflowlog - cannot bin...,<bm>ps|workspacevoidpredefinedtype|methoddecla...,veryearly,monitor - operational insights| customer-reported,2020-12-06


In [12]:
# Cross Validation in TimeSeries
totalNoOfRecords = len(totalData)
split = int(totalNoOfRecords / 5)  
print(totalNoOfRecords, split)

2540 508


In [13]:
# (*func1*) Cleandata
from nltk import PorterStemmer, word_tokenize
from nltk.corpus import stopwords
import regex as re
# 将title_description更新为只保留大小写字母，分词， 去停用词， 去掉单个字母，提取词干后
def Cleandata(data):
    porter = PorterStemmer() # 词干提取器 eg. coding -> code
    for ind in data.index:   # index是默认的4466，4579，...
        # 找到ind的题目描述，只保留字母，word_tokenize分词，去掉stopwords，去掉单字母，提取词干，替换ind的题目描述
        bug_description = data['Title_Description'][ind]
        bug_description = re.sub("[^a-zA-Z]", " ", str(bug_description)) #仅保留字母 /re.sub()
        if bug_description and bug_description.strip():
            tk_bug_description = word_tokenize(bug_description) # 分词 /word_tokenize 
            tokens_without_sw = str(
                [word for word in tk_bug_description if not word in stopwords.words('english')]).strip('[]').replace(
                "'", "").replace(",", "").replace("\\", "") # 去掉 /stopwords 
            tokens_without_sl = ' '.join(
                [w for w in tokens_without_sw.split() if len(w) > 1])  # token without single letter
            tokens_with_stemm = ' '.join([porter.stem(w) for w in tokens_without_sl.split()])  # token with stemm
            data['Title_Description'][ind] = tokens_with_stemm
    return data

In [14]:
# (*func5*) CreateOversamplingWithDataAugmentation
import nlpaug.augmenter.word as naw
import gc
# 将少数列的行数通过bert上下文替换增加到和多数列一样多
def CreateOversamplingWithDataAugmentation(data, splitno):
    # /*Get Contextual Word Embedding Model*/
    aug = naw.ContextualWordEmbsAug(model_path='bert-base-uncased', action="substitute") # aug是上下文替换器
    # /*Get Developer Bug Count/
    dfcountbybug = data.groupby(["Name", "FixedByID"], as_index=True)["FixedByID"].size().reset_index(name="count")
    dfcountbybug.to_csv('list.csv')
    # /*Get Majority Class List*/
    majoritycount = dfcountbybug[(dfcountbybug['FixedByID'] != 'unknown') & (dfcountbybug['Name'] != 'unknown')][
        'count'].max()
    # /*Get Minority Class Count*/
    minoritylist = dfcountbybug[(dfcountbybug['count'] != dfcountbybug['count'].max())]
    # 21 1384 29064
    print(majoritycount, len(minoritylist), majoritycount * len(minoritylist))
    estimatetotalnoofdataaugrecord = majoritycount * len(minoritylist)
    maxnoofaug = majoritycount
    ## Data Aug Record Count Validation, If over threshold reduce the majaritycount
    if estimatetotalnoofdataaugrecord > DataAugThreshold: # 30000
        print('Overtheshold--')
        maxnoofaug = int((DataAugThreshold / estimatetotalnoofdataaugrecord) * majoritycount)

    # /*Loop through Minor Class Group*/
    for ind in minoritylist.index:
        # print(minoritylist['FixedByID'][ind], minoritylist['count'][ind])
        developer = minoritylist['FixedByID'][ind]
        bugtype = minoritylist['Name'][ind]
        minoritycount = minoritylist['count'][ind]
        data1 = data[(data['FixedByID'] == developer) & (data['Name'] == bugtype)]
        # print(len(data1), developer,bugtype)
        # print('minoritycount  --->',minoritycount, 'majoritycount--->',majoritycount, 'index --->', ind , 'out of ', len(minoritylist.index))
        # Create Sample Data until minority class count Match up with majority class count
        while minoritycount < maxnoofaug:
            # majoritycount:
            samplerow = data1.sample()
            oldbugdescription = str(samplerow['Title_Description'].values).strip('[]').replace("'", "")
            if oldbugdescription:
                first100words_aug = str(aug.augment(' '.join(oldbugdescription.split()[:100])))
                remainingwords = ' '.join(oldbugdescription.split()[100:])
                newbugdescription = first100words_aug + remainingwords
                new_row = {'RepoID': str(samplerow['RepoID'].values).strip('[]').replace("'", ""),
                            # 'PullRequestID' : str(samplerow['PullRequestID'].values).strip('[]').replace("'",""),
                           'IssueID': str(samplerow['IssueID'].values).strip('[]').replace("'", ""),
                           'Title_Description': newbugdescription,  # /*Data Augmentation : Title_Desciption*/
                           'AST': str(samplerow['AST'].values).strip('[]').replace("'", ""),
                           'FixedByID': str(samplerow['FixedByID'].values).strip('[]').replace("'", ""),
                           'Name': str(samplerow['Name'].values).strip('[]').replace("'", ""),
                           'CreatedDate': str(samplerow['CreatedDate'].values).strip('[]').replace("'", "")}
                data = data.append(new_row, ignore_index=True)
                minoritycount = minoritycount + 1
            gc.collect()
    trainfilename = DataFileName + 'trainaugdata' + splitno + FileType
    data.to_csv(trainfilename)
    return data

In [15]:
import numpy as np
# (*func7*) one_hot_encode
def one_hot_encode(tags, mapping):
    # create empty vector
    encoding = np.zeros(len(mapping), dtype='uint8')
    # mark 1 for each tag in the vector
    for tag in tags:
        encoding[mapping[tag]] = 1
    return encoding

In [16]:
# (*func6*) create_tag_mapping -> 7
# combineddata_dev = train_dev_y + test_dev_y, 'FixedByID'
def create_tag_mapping(mapping_csv, tagname):
    # tagname FixedByID
    print('tagname', tagname)
    # create a set of all known tags
    labels = set()
    IssueType_Tags = []
    for i in range(len(mapping_csv)):
        # convert spaced separated tags into an array of tags
        tags = mapping_csv[i].split('|')
        # add tags to the set of known labels
        labels.update(tags)
    # labels 存 tag(dev) 
    # convert set of labels to a list to list
    labels = list(labels)
    # order set alphabetically
    labels.sort()
    # label和数字的正反映射
    # dict that maps labels to integers, and the reverse
    labels_map = {labels[i]: i for i in range(len(labels))}
    inv_labels_map = {i: labels[i] for i in range(len(labels))}
    # mapping_csv转为onehot编码
    for i in range(len(mapping_csv)):
        # Create One Hot Encoding For Issue Type
        IssueType_Tag = one_hot_encode(mapping_csv[i].split('|'), labels_map) # (*func7*) one_hot_encode
        IssueType_Tags.append(IssueType_Tag)

    result = IssueType_Tags
    return labels_map, inv_labels_map, result

In [17]:
# (*func8*) RemoveTestRecordIfNotExistInTrainData
def RemoveTestRecordIfNotExistInTrainData(traindata, testdata):
    traingroup = traindata.groupby(["Name", "FixedByID"], as_index=True)["FixedByID"].size().reset_index(name="count")
    testgroup = testdata.groupby(["Name", "FixedByID"], as_index=True)["FixedByID"].size().reset_index(name="count")
    for ind in testgroup.index:
        try:
            record = traindata[
                traindata['FixedByID'].str.match(testgroup['FixedByID'][ind]) & traindata['Name'].str.match(
                    testgroup['Name'][ind])]
            if len(record) < 1:
                print('remove from testdata...')
                testdata = testdata.drop(testdata[
                                             testdata['FixedByID'].str.match(testgroup['FixedByID'][ind]) & testdata[
                                                 'Name'].str.match(testgroup['Name'][ind])].index)
        except:
            print("An exception occurred index :", ind)
    return testdata

In [16]:
from keras.preprocessing.text import Tokenizer
# (*func2*) SplitTrainTestDataForMultiLabelWithDataAugmentation -> 5, 6
def SplitTrainTestDataForMultiLabelWithDataAugmentation(data, splitno):
    # dev -> Developer, btype -> BugType
    x_train_context, x_train_AST, dev_y_train, btype_y_train = [], [], [], []
    x_test_context, x_test_AST, dev_y_test, btype_y_test = [], [], [], []
    tk_context = []

    if LoadDataAugFromFile == True: # false
        traindata = pd.read_csv(DataFilePath + 'DataAugmentation/' + DataFileName + 'trainaugdata' + splitno + FileType,
                                error_bad_lines=False, index_col=False, dtype='unicode', encoding='latin-1',
                                low_memory=False).sample(frac=1)
        traindata = traindata.rename(columns={'ï»¿RepoID': 'RepoID'}, inplace=False)

        testdata = pd.read_csv(DataFilePath + 'DataAugmentation/' + DataFileName + 'testdata' + splitno + FileType,
                               error_bad_lines=False, index_col=False, dtype='unicode', encoding='latin-1',
                               low_memory=False).sample(frac=1)
        testdata = testdata.rename(columns={'ï»¿RepoID': 'RepoID'}, inplace=False)
        testdata = RemoveTestRecordIfNotExistInTrainData(traindata, testdata)
    else:
        # 80% / 20% train / test split:
        train_size = int(len(data) * .8)
        traindata = data[:train_size]
        testdata = data[train_size:]
        # testdata存入DataFileName + 'testdata' + splitno + FileType
        # Save Data to TestFile
        testfilename = DataFileName + 'testdata' + splitno + FileType
        testdata.to_csv(testfilename)
        # /*Create Sample Data*/  (*func5*)CreateOversamplingWithDataAugmentation
        # 增强后traindata存入(DataFileName + 'trainaugdata' + splitno + FileType）
        traindata = CreateOversamplingWithDataAugmentation(traindata, splitno) 

    # /*****************Train Data***********************/
    train_dev_y = list(traindata['FixedByID'])  # Developer List
    train_btype_y = list(traindata['Name'])  # Bug Type List
    train_x_context = list(traindata['Title_Description'])
    traindata.AST = traindata.AST.astype(str)
    train_x_AST = list(traindata['AST'])

    x_train_context = train_x_context
    x_train_AST = train_x_AST

    # /****************Test Data************************/
    x_test_context = list(testdata['Title_Description'])
    x_test_AST = list(testdata['Title_Description'])
    test_dev_y = list(testdata['FixedByID'])  # Developer List
    test_btype_y = list(testdata['Name'])  # Bug Type List

    # /*******************Label Encoder***********************/
    ### Developer Encoder 
    combineddata_dev = train_dev_y + test_dev_y
    # (*func6*) create_tag_mapping 把train_dev_y 和 test_dev_y 由str进行onehot encoder
    dev_labels_map, dev_inv_labels_map, combineddata_dev_enc = create_tag_mapping(combineddata_dev, 'FixedByID')
    dev_y_train = combineddata_dev_enc[:len(train_dev_y)]
    dev_y_test = combineddata_dev_enc[len(train_dev_y):]
    # 31929 1430 33359
    print("Developer", "Training: ", len(train_dev_y), "Testing :", len(test_dev_y), "Combined DEV + TEST",
          len(combineddata_dev_enc))

    ### BugType Encoder
    combineddata_bugtype = train_btype_y + test_btype_y
    # 同理编码bugtype
    btype_labels_map, btype_inv_labels_map, combineddata_bugtype_enc = create_tag_mapping(combineddata_bugtype, 'Name')
    btype_y_train = combineddata_bugtype_enc[:len(train_btype_y)]
    btype_y_test = combineddata_bugtype_enc[len(train_btype_y):]
    print("Bug Type", "Training: ", len(btype_y_train), "Testing :", len(btype_y_test), "Combined DEV + TEST",
          len(combineddata_bugtype_enc))

    # /*******************Tokenizer****************************/

    x_train_context = [str(row).lower() for row in x_train_context]
    x_test_context = [str(row).lower() for row in x_test_context]
    combineddata = x_train_context + x_test_context
    # 不限词汇表大小，以单词为单位，未知词标为Unknown，过滤掉'!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n\d+'
    tk_context = Tokenizer(num_words=None, char_level=None, oov_token='Unknown',
                           filters='!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n\d+')
    tk_context.fit_on_texts(combineddata)

    return (traindata, # traindata = CreateOversamplingWithDataAugmentation(traindata, splitno)
            tk_context, 
            x_train_context, x_train_AST, dev_y_train, btype_y_train, 
            x_test_context, x_test_AST, dev_y_test, btype_y_test, 
            dev_labels_map, dev_inv_labels_map, btype_labels_map, btype_inv_labels_map)

In [17]:
from keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
# (*func4*) Tokenize
def Tokenize(tk_context, 
             x_train_context, x_train_AST, x_test_context, x_test_AST):
    # =======================Convert string to index================
    ## context 小写后 转为 toeknizer 
    # Tokenizer
    x_train_context = [str(row).lower() for row in x_train_context]
    x_test_context = [str(row).lower() for row in x_test_context]
    combineddata_context = x_train_context + x_test_context
    ## 不限词汇表大小，以单词为单位，未知词标为Unknown，过滤掉'!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n\d+'
    tk_context = Tokenizer(num_words=None, char_level=None, oov_token='Unknown',
                           filters='!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n')
    tk_context.fit_on_texts(combineddata_context)

    ## AST 小写后 转为 toeknizer 
    x_train_AST = [str(row).lower() for row in x_train_AST]
    x_test_AST = [str(row).lower() for row in x_test_AST]
    combineddata_AST = x_train_AST + x_test_AST
    tk_AST = Tokenizer(num_words=None, char_level=None, oov_token='Unknown')
    tk_AST.fit_on_texts(combineddata_AST)

    # Convert string to index 
    x_train_context_sequences = tk_context.texts_to_sequences(x_train_context)
    x_train_AST_sequences = tk_AST.texts_to_sequences(x_train_AST)
    x_test_context_sequences = tk_context.texts_to_sequences(x_test_context)
    x_test_AST_sequences = tk_AST.texts_to_sequences(x_test_AST)

    # Padding
    x_train_context = pad_sequences(x_train_context_sequences, maxlen=MAX_SEQUENCE_LENGTH, padding='post')
    x_train_AST = pad_sequences(x_train_AST_sequences, maxlen=MAX_SEQUENCE_LENGTH, padding='post')
    x_test_context = pad_sequences(x_test_context_sequences, maxlen=MAX_SEQUENCE_LENGTH, padding='post')
    x_test_AST = pad_sequences(x_test_AST_sequences, maxlen=MAX_SEQUENCE_LENGTH, padding='post')

    # Convert to numpy array
    x_train_context = np.array(x_train_context)
    x_train_AST = np.array(x_train_AST)
    x_test_context = np.array(x_test_context)
    x_test_AST = np.array(x_test_AST)
    return tk_context, tk_AST, x_train_context, x_train_AST, x_test_context, x_test_AST

In [ ]:
 # (*func3*) SplitTrainTestData
def SplitTrainTestData(data):
    # dev -> Developer, btype -> BugType
    x_train_context, x_train_AST, dev_y_train, btype_y_train = [], [], [], []
    x_test_context, x_test_AST, dev_y_test, btype_y_test = [], [], [], []
    tk_context = []

    dev_y = list(data['FixedByID'].astype(str))  # Developer List
    btype_y = list(data['Name'].astype(str))  # Bug Type List
    data.Title_Description = data.Title_Description.astype(str)
    x_context = list(data['Title_Description'])
    data.AST = data.AST.astype(str)
    x_AST = list(data['AST'])

    # 80% / 20% train / test split:
    train_size = int(len(x_context) * .8)

    x_train_context = x_context[:train_size]
    x_train_AST = x_AST[:train_size]
    dev_y_train = dev_y[:train_size]
    btype_y_train = btype_y[:train_size]

    x_test_context = x_context[train_size:]
    x_test_AST = x_AST[train_size:]
    dev_y_test = dev_y[train_size:]
    btype_y_test = btype_y[train_size:]

    # /*******************Label Encoder***********************/
    ### Developer Encoder
    combineddata_dev = dev_y_train + dev_y_test
    dev_labels_map, dev_inv_labels_map, combineddata_dev_enc = create_tag_mapping(combineddata_dev, 'FixedByID')
    dev_y_train = combineddata_dev_enc[:len(dev_y_train)]
    dev_y_test = combineddata_dev_enc[len(dev_y_train):]
    print("Developer", "Training: ", len(dev_y_train), "Testing :", len(dev_y_test), "Combined DEV + TEST",
          len(combineddata_dev_enc))

    ### BugType Encoder
    combineddata_bugtype = btype_y_train + btype_y_test
    btype_labels_map, btype_inv_labels_map, combineddata_bugtype_enc = create_tag_mapping(combineddata_bugtype, 'Name')
    btype_y_train = combineddata_bugtype_enc[:len(btype_y_train)]
    btype_y_test = combineddata_bugtype_enc[len(btype_y_train):]
    print("Bug Type", "Training: ", len(btype_y_train), "Testing :", len(btype_y_test), "Combined DEV + TEST",
          len(combineddata_bugtype_enc))

    # Tokenizer
    x_train_context = [str(row).lower() for row in x_train_context]
    x_test_context = [str(row).lower() for row in x_test_context]
    combineddata = x_train_context + x_test_context
    tk_context = Tokenizer(num_words=None, char_level=None, oov_token='Unknown',
                           filters='!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n\d+')
    tk_context.fit_on_texts(combineddata)

    return tk_context, x_train_context, x_train_AST, dev_y_train, btype_y_train, x_test_context, x_test_AST, dev_y_test, btype_y_test, dev_labels_map, dev_inv_labels_map, btype_labels_map, btype_inv_labels_map

In [ ]:
# 5-folds时序CV
for i in [5]:
    currentSplit = split * i
    data = totalData[:int(currentSplit)]
    Cleandata(data) # Cleandata函数(*func1*)
    # 判断数据要不要用SplitTrainTestDataForMultiLabelWithDataAugmentation函数(*func2*)增强
    # 还是直接用SplitTrainTestData函数（*func3）分
    if DataAugmentation == True: 
        (traindata, 
         tk_context, 
         x_train_context, x_train_AST, dev_y_train, btype_y_train, 
         x_test_context, x_test_AST, dev_y_test, btype_y_test, 
         dev_labels_map, dev_inv_labels_map, btype_labels_map, btype_inv_labels_map) = \
         (SplitTrainTestDataForMultiLabelWithDataAugmentation(data, str(i)))
    else:
        (tk_context, 
         x_train_context, x_train_AST, dev_y_train, btype_y_train, 
         x_test_context, x_test_AST, dev_y_test, btype_y_test, 
         dev_labels_map, dev_inv_labels_map, btype_labels_map, btype_inv_labels_map) = \
        (SplitTrainTestData(data))
    # 这里这个Tokenize函数(*func4*)来tokenize并转为np.array
    (tk_context, tk_AST, 
     x_train_context, x_train_AST, 
     x_test_context, x_test_AST) = Tokenize(tk_context, x_train_context,
                                            x_train_AST, x_test_context,
                                            x_test_AST)
    print('finished iteration', i)

/tmp/ipykernel_1161/2374138475.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['Title_Description'][ind] = tokens_with_stemm
/tmp/ipykernel_1161/2262987944.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Cleandata(data) # Cleandata函数(*func1*)
